I will show you the originial codes first

In [2]:
# Import packages
import pandas as pd
import datetime

# read data
url2= "https://covid19-static.cdn-apple.com/covid19-mobility-data/2013HotfixDev18/v3/en-us/applemobilitytrends-2020-08-03.csv"
apple_mobility = pd.read_csv(url2,header=0)

# select data for Los Angeles
apple_mobility = apple_mobility[apple_mobility['region']=='Los Angeles']

# Process data into format
amobility= apple_mobility.copy()
amobility = amobility.set_index('transportation_type').transpose().iloc[5:]

amobility['date'] = pd.to_datetime(amobility.index)
amobility

transportation_type,driving,transit,walking,date
2020-01-13,100,100,100,2020-01-13
2020-01-14,105.93,103.78,105.72,2020-01-14
2020-01-15,112.23,105.28,111.35,2020-01-15
2020-01-16,111.73,103.12,111.68,2020-01-16
2020-01-17,130.2,109.34,134.45,2020-01-17
...,...,...,...,...
2020-07-30,109.82,46.04,111.63,2020-07-30
2020-07-31,124.8,49.42,127.5,2020-07-31
2020-08-01,116.76,42.81,132.95,2020-08-01
2020-08-02,95.16,38.03,105.03,2020-08-02


In [3]:
amobility.info()

<class 'pandas.core.frame.DataFrame'>
Index: 204 entries, 2020-01-13 to 2020-08-03
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   driving  202 non-null    object        
 1   transit  202 non-null    object        
 2   walking  202 non-null    object        
 3   date     204 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 8.0+ KB


__Now, I can show you the codes I found in Git__

In [4]:
# import packages

import os
import datetime
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re
import json

import pandas as pd
import zipfile as zp

In [5]:
# three functions for data collection

def get_apple_link():
    '''Get link of Apple Mobility Trends report file
    
       Returns:
           link (str): link of Apple Mobility Trends report file
    '''
    # get link via API
    json_link = "https://covid19-static.cdn-apple.com/covid19-mobility-data/current/v3/index.json"
    with urllib.request.urlopen(json_link) as url:
        json_data = json.loads(url.read().decode())
    link = "https://covid19-static.cdn-apple.com" + \
        json_data['basePath'] + json_data['regions']['en-us']['csvPath']
    return link

def download_apple_report(directory="apple_reports"):
    '''Download Apple Mobility Trends report in CSV

        Args:
            directory: directory to which CSV report will be downloaded

        Returns:
            new_files (bool): flag indicating whether or not a new file has been downloaded
    '''
    new_files = False
    
    # create directory if it don't exist
    if not os.path.exists(directory) and directory!='':
        os.makedirs(directory)
    
    link = get_apple_link()
    file_name = "applemobilitytrends.csv"
    path = os.path.join(directory, file_name)
    path = os.path.join(directory, file_name)
    old_size = os.path.getsize(path) if os.path.isfile(path) else 0
    urllib.request.urlretrieve(link, path)
    new_size = os.path.getsize(path)
    if old_size!=new_size:
        new_files = True

    if not new_files:
        print('Apple: No updates')
    else:
        print('Apple: Update available')
    
    return new_files

def build_apple_report(
    source=os.path.join(
        'apple_reports',
        "applemobilitytrends.csv"),
        report_type="regions"):
    '''Build cleaned Apple report (transform dates from columns to rows, add country names for subregions and cities)
       for the worldwide or for some country (currently only for the US)

        Args:
            source: location of the raw Apple CSV report
            destination: destination file path
            report_type: two options available: "regions" - report for the worldwide, "US" - report for the US

        Returns:
           apple (DataFrame): generated Apple report
    '''
    apple = pd.read_csv(source)
    apple = apple.drop(columns=['alternative_name'])
    apple['country'] = apple.apply(
        lambda x: x['region'] if x['geo_type'] == 'country/region' else x['country'],
        axis=1)

    if report_type == 'regions':
        apple = apple[apple.geo_type != 'county']
        apple['sub-region'] = apple.apply(lambda x: 'Total' if x['geo_type'] == 'country/region' else (
            x['region'] if x['geo_type'] == 'sub-region' else x['sub-region']), axis=1)
        apple['subregion_and_city'] = apple.apply(
            lambda x: 'Total' if x['geo_type'] == 'country/region' else x['region'], axis=1)
        apple = apple.drop(columns=['region'])
        apple['sub-region'] = apple['sub-region'].fillna(
            apple['subregion_and_city'])

        apple = apple.melt(
            id_vars=[
                'geo_type',
                'subregion_and_city',
                'sub-region',
                'transportation_type',
                'country'],
            var_name='date')
        apple['value'] = apple['value'] - 100

        apple = apple.pivot_table(
            index=[
                "geo_type",
                "subregion_and_city",
                "sub-region",
                "date",
                "country"],
            columns='transportation_type').reset_index()
        apple.columns = [t + (v if v != "value" else "")
                         for v, t in apple.columns]
        apple = apple.loc[:,
                          ['country',
                           'sub-region',
                           'subregion_and_city',
                           'geo_type',
                           'date',
                           'driving',
                           'transit',
                           'walking']]
        apple = apple.sort_values(by=['country',
                                      'sub-region',
                                      'subregion_and_city',
                                      'date']).reset_index(drop=True)
    elif report_type == "US":
        apple = apple[apple.country == "United States"].drop(columns=[
                                                             'country'])
        apple['sub-region'] = apple['sub-region'].fillna(
            apple['region']).replace({"United States": "Total"})
        apple['region'] = apple.apply(lambda x: x['region'] if (
            x['geo_type'] == 'city' or x['geo_type'] == 'county') else 'Total', axis=1)
        apple = apple.rename(
            columns={
                'sub-region': 'state',
                'region': 'county_and_city'})

        apple = apple.melt(
            id_vars=[
                'geo_type',
                'state',
                'county_and_city',
                'transportation_type'],
            var_name='date')
        apple['value'] = apple['value'] - 100

        apple = apple.pivot_table(
            index=[
                'geo_type',
                'state',
                'county_and_city',
                'date'],
            columns='transportation_type').reset_index()
        apple.columns = [t + (v if v != "value" else "")
                         for v, t in apple.columns]

        apple = apple.loc[:, ['state', 'county_and_city', 'geo_type',
                              'date', 'driving', 'transit', 'walking']]
        apple = apple.sort_values(
            by=['state', 'county_and_city', 'geo_type', 'date']).reset_index(drop=True)
    return apple

In [6]:
# Use functions to get the data
new_files_status_apple = download_apple_report()
apple_mobility = build_apple_report(report_type="US")
apple_mobility

# Select data for Los Angeles
apple_mobility = apple_mobility[apple_mobility['county_and_city']=='Los Angeles']
apple_mobility['driving'] = apple_mobility['driving']+100
apple_mobility['transit'] = apple_mobility['transit']+100
apple_mobility['walking'] = apple_mobility['walking']+100

apple_mobility

Apple: No updates


,state,county_and_city,geo_type,date,driving,transit,walking
30906,California,Los Angeles,city,2020-01-13,100.00,100.00,100.00
30907,California,Los Angeles,city,2020-01-14,105.93,103.78,105.72
30908,California,Los Angeles,city,2020-01-15,112.23,105.28,111.35
30909,California,Los Angeles,city,2020-01-16,111.73,103.12,111.68
30910,California,Los Angeles,city,2020-01-17,130.20,109.34,134.45
...,...,...,...,...,...,...,...
31103,California,Los Angeles,city,2020-07-30,109.82,46.04,111.63
31104,California,Los Angeles,city,2020-07-31,124.80,49.42,127.50
31105,California,Los Angeles,city,2020-08-01,116.76,42.81,132.95
31106,California,Los Angeles,city,2020-08-02,95.16,38.03,105.03


In [9]:
# Format them
amobility = apple_mobility.copy()
amobility = amobility[['date','driving','transit','walking']]

amobility['date'] = pd.to_datetime(amobility['date'])
amobility

,date,driving,transit,walking
30906,2020-01-13,100.00,100.00,100.00
30907,2020-01-14,105.93,103.78,105.72
30908,2020-01-15,112.23,105.28,111.35
30909,2020-01-16,111.73,103.12,111.68
30910,2020-01-17,130.20,109.34,134.45
...,...,...,...,...
31103,2020-07-30,109.82,46.04,111.63
31104,2020-07-31,124.80,49.42,127.50
31105,2020-08-01,116.76,42.81,132.95
31106,2020-08-02,95.16,38.03,105.03


In [10]:
amobility.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 202 entries, 30906 to 31107
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     202 non-null    datetime64[ns]
 1   driving  202 non-null    float64       
 2   transit  202 non-null    float64       
 3   walking  202 non-null    float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 7.9 KB
